In [ ]:
!pip install langchain langchain-community langchain-nvidia-ai-endpoints

In [ ]:
!pip install pinecone-client langchain-pinecone

In [ ]:
!pip install langchain-text-splitters

In [ ]:
!pip install pypdf

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import PromptTemplate
from langchain_classic import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

In [3]:
import os
os.environ["NVIDIA_API_KEY"] ="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"

In [4]:
embbedings=NVIDIAEmbeddings()
embedde_query=embbedings.embed_query("heelo how are you")
print(len(embedde_query))

1024


In [6]:
pdf=PyPDFLoader("/content/medical_diet_guidelines.pdf")
data=pdf.load()

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=100)

In [8]:
texts=text_splitter.split_documents(data)

In [9]:
os.environ['PINECONE_API_KEY']="pcsk_3vZeYq_RAbo33xc1RRwKMi2NySLySEd1rxzW7sA5k5S2nb2fqWJPPhWx2LfBiysrDS8bt1"
index_name="diet-plan"

In [10]:
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "diet-plan"

if index_name not in [i.name for i in pc.list_indexes()]:
    raise ValueError("Index does not exist. Create it first.")


In [11]:
len(texts)

3273

In [ ]:
from langchain_core import documents
from langchain_pinecone import PineconeVectorStore

vector_store=PineconeVectorStore.from_documents(
    documents=texts,
    embedding=embbedings,
    index_name=index_name
)

In [ ]:
results=vector_store.similarity_search(
    query="what is diet for hypertension",
    k=3
)
print(results)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


llm = ChatNVIDIA(
    model="meta/llama-3-8b-instruct",
    temperature=0
)


retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


prompt = PromptTemplate.from_template(
"""
You are a medical diet recommendation assistant.

STRICT RULES:
- Use ONLY the CONTEXT provided.
- Do NOT hallucinate calorie values or nutrients.
- If exact values are not present in the context, provide reasonable ranges and clearly mention they are approximate.
- If information is missing, say "I don't know".

TASK:
The disease is predicted by an ML model.
You must:
1. State the predicted disease.
2. Briefly explain the disease.
3. Explain why this diet helps the disease.
4. Generate a ONE-DAY diet plan with detailed food information.

OUTPUT MUST BE VALID JSON ONLY.

JSON FORMAT:
{{
  "predicted_disease": "<disease_name>",
  "disease_summary": "<short explanation of the disease>",
  "diet_goal": "<main dietary goal such as low sodium / low sugar / iron rich>",
  "diet_plan": {{
    "morning": [
      {{
        "food": "<food name>",
        "portion": "<quantity>",
        "approx_calories": "<calories or range>",
        "nutritional_benefit": "<why this food helps the condition>"
      }}
    ],
    "afternoon": [
      {{
        "food": "<food name>",
        "portion": "<quantity>",
        "approx_calories": "<calories or range>",
        "nutritional_benefit": "<why this food helps the condition>"
      }}
    ],
    "evening": [
      {{
        "food": "<food name>",
        "portion": "<quantity>",
        "approx_calories": "<calories or range>",
        "nutritional_benefit": "<why this food helps the condition>"
      }}
    ],
    "night": [
      {{
        "food": "<food name>",
        "portion": "<quantity>",
        "approx_calories": "<calories or range>",
        "nutritional_benefit": "<why this food helps the condition>"
      }}
    ]
  }},
  "daily_estimated_calories": "<approx total calories or range>",
  "foods_to_avoid": [],
  "medical_note": "This is not a substitute for professional medical advice."
}}

CONTEXT:
{context}

QUESTION:
{question}

ANSWER (JSON ONLY):
"""
)



def format_docs(docs):
    if not docs:
        return "NO RELEVANT MEDICAL CONTEXT FOUND"
    return "\n\n".join(doc.page_content[:700] for doc in docs)


rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)


response = rag_chain.invoke(
    "Disease predicted by ML model: Diabetics. Generate diet plan."
)

print(response.content)
